In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
import gymnasium as gym

from astropy.io import fits
from datetime import datetime, timezone
import json
import fitsio
import pandas as pd
import time
import pickle
import re

%reload_ext autoreload
%autoreload 2

In [2]:
import survey_ops
from survey_ops.utils import units, geometry, interpolate
from survey_ops.src.offline_dataset import OfflineDECamDataset
from survey_ops.src.agents import Agent
from survey_ops.src.algorithms import DDQN, BehaviorCloning
from survey_ops.utils.pytorch_utils import seed_everything


In [3]:
SEED = 10
seed_everything(SEED)
torch.set_default_dtype(torch.float32)
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"   
)

In [9]:
from survey_ops.utils.config import Config

In [56]:
cfg = Config('../experiment_results/default_config.json')

In [72]:
with open(cfg.get('paths')['FIELD2NVISITS'], 'r') as f:
    field2nvisits = json.load(f)
with open(cfg.get('paths')['FIELD2NAME'], 'r') as f:
    field2name = json.load(f)
with open(cfg.get('paths')['FIELD2RADEC'], 'r') as f:
    field2radec = json.load(f)

In [74]:
from survey_ops.utils.script_utils import load_raw_data_to_dataframe

In [75]:
raw_data_df = load_raw_data_to_dataframe(cfg.get('paths.DFITS'), cfg.get('paths.DJSON'))


In [78]:
nvisits = list(field2nvisits.values())

In [79]:
max_visited_field = str(np.argmax(nvisits))
max_visited_field

'6381'

In [80]:
field2nvisits[max_visited_field]

25

In [103]:
np.mean(nvisits)

np.float64(5.137574056750857)

In [104]:
np.median(nvisits)

np.float64(5.0)

In [105]:
np.std(nvisits)

np.float64(1.8629118037565457)

In [81]:
field2radec[max_visited_field]

[6.072430937091248, -0.7279120981411417]

In [82]:
field2name[max_visited_field]

'DES survey hex -122-420 tiling 5'

In [90]:
max_obj_df = raw_data_df[raw_data_df['object'].str.contains(field2name[max_visited_field])]

# Load and cut data from fits file, save as .json

In [4]:
data_json_fn = '../data/decam-exposures-20251211.json'
data_fits_fn = '../data/decam-exposures-20251211.fits'
data_json_full_fn = '../data/decam-exposures-20251211-full.json'


In [5]:
try:
    # --- Load json df ---- #
    df = pd.read_json(data_json_fn)
    print('Loaded data from json')
except:
    # --- Load fits ---- #
    print("No .json file found. Instead loading and processing .fits file.")
    d = fitsio.read(data_fits_fn)
    sel = (d['propid'] == '2012B-0001') & (d['exptime'] > 40) & (d['exptime'] < 100) & (~np.isnan(d['teff']))
    selected_d = d[sel]
    column_names = selected_d.dtype.names
    df = pd.DataFrame(selected_d, columns=column_names)
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
    
    df.to_json(data_json_fn, indent=2)
    df = pd.read_json(data_json_fn)
    

Loaded data from json


# Instantiate the offline dataset choosing binning method (healpix vs uniform_grid)

In [11]:
binning_method='healpix'
nside=16
bin_space='radec'
specific_years=None
specific_months = None
specific_days=None
# specific_years=[2016]
# specific_months=[9]
# specific_days=[15]
include_default_features=True
include_bin_features=False
do_z_score_norm=False
do_cyclical_norm=True
do_max_norm=True
do_inverse_airmass=True

In [12]:
train_dataset = OfflineDECamDataset(
    df, 
    binning_method=binning_method,
    nside=nside,
    bin_space=bin_space,
    specific_years=specific_years,
    specific_months=specific_months,
    specific_days=specific_days,
    include_default_features=include_default_features,
    include_bin_features=include_bin_features,
    do_z_score_norm=do_z_score_norm,
    do_cyclical_norm=do_cyclical_norm,
    do_max_norm=do_max_norm,
    do_inverse_airmass=do_inverse_airmass,
    calculate_action_mask=False
)

Calculating zenith states: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 648/648 [00:03<00:00, 198.52it/s]
